# Corporación Favorita Grocery Sales Forecasting
https://www.kaggle.com/c/favorita-grocery-sales-forecasting/data

Brick-and-mortar grocery stores are always in a delicate dance with purchasing and sales forecasting. Predict a little over, and grocers are stuck with overstocked, perishable goods. Guess a little under, and popular items quickly sell out, leaving money on the table and customers fuming.

The problem becomes more complex as retailers add new locations with unique needs, new products, ever transitioning seasonal tastes, and unpredictable product marketing. Corporación Favorita, a large Ecuadorian-based grocery retailer, knows this all too well. They operate hundreds of supermarkets, with over 200,000 different products on their shelves.

Corporación Favorita has challenged the Kaggle community to build a model that more accurately forecasts product sales. They currently rely on subjective forecasting methods with very little data to back them up and very little automation to execute plans. They’re excited to see how machine learning could better ensure they please customers by having just enough of the right products at the right time.

Despendent variable is - How many units of each kind of product were sold in each store on each day during the two-week period

## Imports

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [3]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics
import gc

In [8]:
PATH = "data/"

In [10]:
!ll {PATH}

/bin/sh: 1: ll: not found


In [5]:
!head -5 data/train.csv
!tail -3 data/train.csv

id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2013-01-01,25,103665,7.0,
1,2013-01-01,25,105574,1.0,
2,2013-01-01,25,105575,2.0,
3,2013-01-01,25,108079,1.0,
125497037,2017-08-15,54,2110456,192.0,False
125497038,2017-08-15,54,2113914,198.0,True
125497039,2017-08-15,54,2116416,2.0,False


In [6]:
types={'id':'int64',
       'store_nbr':'int8', 
       'item_nbr':'int32',
       'unit_sales':'float32',
       'onpromotion':'object'}

In [11]:
%%time
df_all = pd.read_csv(f'{PATH}train.csv', parse_dates = ['date'], dtype = types, 
                     infer_datetime_format = True)

CPU times: user 1min 30s, sys: 4.28 s, total: 1min 34s
Wall time: 1min 35s


In [12]:
df_all.onpromotion.fillna(False, inplace = True)
df_all.onpromotion = df_all.onpromotion.map({'False': False, 'True' : True})
df_all.onpromotion = df_all.onpromotion.astype(bool)

In [11]:
%time df_all.describe(include='all')

CPU times: user 26.2 s, sys: 1.23 s, total: 27.5 s
Wall time: 27.5 s


,id,date,store_nbr,item_nbr,unit_sales,onpromotion
count,1.254970e+08,125497040,1.254970e+08,1.254970e+08,1.254970e+08,125497040
unique,NaN,1684,NaN,NaN,NaN,2
top,NaN,2017-07-01 00:00:00,NaN,NaN,NaN,False
freq,NaN,118194,NaN,NaN,NaN,96028767
first,NaN,2013-01-01 00:00:00,NaN,NaN,NaN,NaN
last,NaN,2017-08-15 00:00:00,NaN,NaN,NaN,NaN
mean,6.274852e+07,NaN,2.746458e+01,9.727692e+05,8.554856e+00,NaN
std,3.622788e+07,NaN,1.633051e+01,5.205336e+05,2.360515e+01,NaN
min,0.000000e+00,NaN,1.000000e+00,9.699500e+04,-1.537200e+04,NaN
25%,3.137426e+07,NaN,1.200000e+01,5.223830e+05,2.000000e+00,NaN


In [ ]:
df_all.to_feather('tmp/raw')

In [12]:
df_all = pd.read_feather('tmp/raw')

In [13]:
df_all.tail()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
125497035,125497035,2017-08-15,54,2089339,4.0,False
125497036,125497036,2017-08-15,54,2106464,1.0,True
125497037,125497037,2017-08-15,54,2110456,192.0,False
125497038,125497038,2017-08-15,54,2113914,198.0,True
125497039,125497039,2017-08-15,54,2116416,2.0,False


In [14]:
df_all.unit_sales = np.log1p(np.clip(df_all.unit_sales, 0, None))

In [16]:
%time add_datepart(df_all, 'date')

MemoryError: 

In [17]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

In [20]:
gc.collect()

0